

# BigData Airlines

A Eleflow irá atender um novo cliente, a _BigData Airlines_, e você será o engenheiro de dados responsável por fazer a ingestão de dados e preparar algumas tabelas para os cientistas de dados e analistas de dados.

## Seção 0: Importar bibliotecas e configurar o ambiente

In [ ]:
# Importe as bibliotecas necessárias
import os
import pandas as pd
import requests
# Configurar as chaves da API (se necessário)
RAPIDAPI_KEY = "https://rapidapi.com/Active-api/api/airport-info/"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Ingestão de Dados

In [ ]:
# Diretório dos arquivos JSON da pasta VRA
vra_dir = '/content/drive/MyDrive/ELEFlow/VRA/'

# Diretório dos arquivos CSV da pasta AIR_CIA
air_cia_dir = '/content/drive/MyDrive/ELEFlow/AIR_CIA/'

# Função para obter informações de aeródromos com base no código ICAO
def get_aerodromo_info(icao_code):
    api_url = f'https://rapidapi.com/Active-api/api/airport-info/{icao_code}'
    headers = {
        'X-RapidAPI-Host': 'active-api',
        'X-RapidAPI-Key': 'your-api-key'  # Substitua pelo seu API Key
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return None

# Carregar Dados de VRA

# Lista para armazenar os DataFrames carregados dos arquivos JSON
vra_dataframes = []

# Percorra os arquivos JSON na pasta VRA e normalize os cabeçalhos
for filename in os.listdir(vra_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(vra_dir, filename)
        df = pd.read_json(file_path)
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        vra_dataframes.append(df)

# Concatene os DataFrames em um único DataFrame
vra_combined = pd.concat(vra_dataframes, ignore_index=True)

# Salve os dados em um formato adequado (por exemplo, CSV)
vra_combined.to_csv('/content/drive/MyDrive/ELEFlow/vra_combined.csv', index=False)

# Carregar Dados da Pasta AIR_CIA

# Lista para armazenar os DataFrames carregados dos arquivos CSV
air_cia_dataframes = []

# Percorra os arquivos CSV na pasta AIR_CIA e normalize os cabeçalhos
for filename in os.listdir(air_cia_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(air_cia_dir, filename)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.lower().str.replace(' ', '_')

        # Divida a coluna 'ICAO IATA' em duas colunas
        df[['ICAO', 'IATA']] = df['icao_iata'].str.split(expand=True)

        air_cia_dataframes.append(df)

# Concatene os DataFrames em um único DataFrame
air_cia_combined = pd.concat(air_cia_dataframes, ignore_index=True)

# Salve os dados em um formato adequado
air_cia_combined.to_csv('/content/drive/MyDrive/ELEFlow/air_cia_combined.csv', index=False)

## Seção 2: Criação de Tabelas de Aeródromos

In [ ]:
# Criar Tabela de Aeródromos

# Crie um DataFrame vazio para armazenar os dados dos aeródromos
aerodromos_df = pd.DataFrame(columns=['ICAO', 'Nome', 'Localizacao', 'Outras_Informacoes'])

# Itere sobre os códigos ICAO dos dados VRA
for icao_code in vra_combined['icao'].unique():
    aerodromo_info = get_aerodromo_info(icao_code)
    if aerodromo_info:
        aerodromos_df = aerodromos_df.append({
            'ICAO': icao_code,
            'Nome': aerodromo_info['nome'],
            'Localizacao': aerodromo_info['localizacao'],
            'Outras_Informacoes': aerodromo_info['outras_informacoes']
        }, ignore_index=True)

# Salve os dados dos aeródromos em um formato adequado
aerodromos_df.to_csv('/content/drive/MyDrive/ELEFlow/aerodromos.csv', index=False)

## Seção 3: Criação da View

In [ ]:
# Criação das Views em SQL

# Importe a biblioteca para SQL no ambiente do Google Colab
from google.colab import sql
sql_conn = sql.Connection()

# View 1: Rota mais utilizada por companhia aérea
view1_query = """
CREATE OR REPLACE VIEW RotaMaisUtilizada AS
SELECT
    c.RazaoSocial AS CompanhiaAerea,
    a1.Nome AS AeroportoOrigem,
    a1.ICAO AS ICAOOrigem,
    a1.Estado AS EstadoOrigem,
    a2.Nome AS AeroportoDestino,
    a2.ICAO AS ICAODestino,
    a2.Estado AS EstadoDestino
FROM CompanhiasAereas c
JOIN Voos v ON c.ID = v.CompanhiaAereaID
JOIN Aeroportos a1 ON v.AeroportoOrigemID = a1.ID
JOIN Aeroportos a2 ON v.AeroportoDestinoID = a2.ID
GROUP BY c.RazaoSocial
ORDER BY COUNT(*) DESC
LIMIT 1;
"""

# View 2: Companhia aérea com maior atuação em um aeroporto por ano
view2_query = """
CREATE OR REPLACE VIEW CompanhiaMaisAtuacao AS
SELECT
    a.Nome AS Aeroporto,
    a.ICAO AS ICAOAeroporto,
    c.RazaoSocial AS CompanhiaAerea,
    COUNT(DISTINCT v1.ID) AS RotasSaindo,
    COUNT(DISTINCT v2.ID) AS RotasChegando,
    COUNT(*) AS TotalPousosDecolagens
FROM Aeroportos a
JOIN Voos v1 ON a.ID = v1.AeroportoOrigemID
JOIN Voos v2 ON a.ID = v2.AeroportoDestinoID
JOIN CompanhiasAereas c ON v1.CompanhiaAereaID = c.ID
GROUP BY a.Nome, c.RazaoSocial
HAVING COUNT(*) = (SELECT MAX(CountTotal) FROM
    (SELECT a1.Nome AS AeroportoNome, c1.RazaoSocial AS CompanhiaNome, COUNT(*) AS CountTotal
     FROM Aeroportos a1
     JOIN Voos v1 ON a1.ID = v1.AeroportoOrigemID
     JOIN CompanhiasAereas c1 ON v1.CompanhiaAereaID = c1.ID
     GROUP BY a1.Nome, c1.RazaoSocial) t
    WHERE t.AeroportoNome = a.Nome);
"""

# Execute as queries SQL no ambiente do Google Colab
sql_conn.execute(view1_query)
sql_conn.execute(view2_query)


## Justificativas sobre Escalabilidade e Camadas



  1.Escalabilidade de Tecnologia: A utilização de Python com bibliotecas como pandas e requests é escalável, pois essas bibliotecas são eficientes para manipular grandes volumes de dados. Além disso, a abordagem de normalização de dados e uso de SQL para criação de visualizações é escalável, pois permite o processamento eficiente de grandes conjuntos de dados.

  2.Camadas de Ingestão de Dados: O processo de ingestão de dados é dividido em várias camadas, incluindo a camada de leitura de arquivos, normalização de dados, obtenção de informações de aeródromos e criação de visualizações. Essas camadas permitem uma organização modular e escalável do processo de ingestão, facilitando futuras expansões e manutenção. O uso de um banco de dados relacional também ajuda na organização dos dados e consultas eficientes.

  3.Automatização: A automação do processo de ingestão, como agendar a execução regular do código, torna a solução escalável para lidar com dados que são atualizados regularmente.